In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

events = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv").withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))
devices = spark.read.option("header","true").csv("/home/iceberg/data/devices.csv")

df = events.join(devices,on="device_id",how="left")
df = df.withColumnsRenamed({'browser_type': 'browser_family', 'os_type': 'os_family'})

df.show()

25/08/20 11:07:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-----------+
| device_id|    user_id|referrer|                host|       url|          event_time|         event_date|browser_family|os_family|device_type|
+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-----------+
| 532630305| 1037710827|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|         Other|    Other|      Other|
| 532630305|  925588856|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|         Other|    Other|      Other|
| 532630305|-1180485268|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|         Other|    Other|      Other|
| 532630305|-1044833855|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|         Other|    Other|     

In [2]:
sorted = df.repartition(10, col("event_date"))\
    .sortWithinPartitions(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sortedTwo = df.repartition(10, col("event_date"))\
    .sort(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.show()
sortedTwo.show()


+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
|  device_id|    user_id|            referrer|                host|                 url|          event_time|         event_date|browser_family|os_family|       device_type|
+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------+---------+------------------+
|  532630305| 1129583063|                NULL|admin.zachwilson....|                   /|2021-01-07 09:21:...|2021-01-07 00:00:00|         Other|    Other|             Other|
| 1088283544| -648945006|                NULL|    www.eczachly.com|                   /|2021-01-07 02:58:...|2021-01-07 00:00:00|      PetalBot|  Android|Generic Smartphone|
| -158310583|-1871780024|                NULL|    www.eczachly.com|                   /|2021-01-07 04:17:...|2021-01-07 00:00:00| 

In [ ]:
# .sortWithinPartitions() sorts within partitions, whereas .sort() is a global sort, which is very slow

# Note - exchange is synonymous with Shuffle

In [3]:
sorted = df.repartition(10, col("event_date"))\
    .sortWithinPartitions(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sortedTwo = df.repartition(10, col("event_date"))\
    .sort(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.explain()
sortedTwo.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#261, event_date#29, browser_family#74, os_family#75, device_type#58]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(event_date#29, 10), REPARTITION_BY_NUM, [plan_id=410]
         +- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, event_time#22, event_date#29, browser_type#56 AS browser_family#74, os_type#57 AS os_family#75, device_type#58]
            +- BroadcastHashJoin [device_id#18], [device_id#55], LeftOuter, BuildRight, false
               :- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
               :  +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilters:

In [4]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [5]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [6]:
%%sql

DROP TABLE IF EXISTS bootcamp.events_sorted

++
||
++
++

In [7]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));


++
||
++
++

In [8]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));

++
||
++
++

In [9]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (year(event_date));

++
||
++
++

In [10]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    
first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("host"))

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [11]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files





size,num_files,sorted
5444946,4,sorted
5556664,4,unsorted


In [17]:
%%sql
SELECT count(1) FROM demo.bootcamp.events_sorted.files;

count(1)
4


In [3]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

count(1)
3665


In [2]:
%%sql 
SELECT * FROM bootcamp.match_details_bucketed.files


content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,referenced_data_file,content_offset,content_size_in_bytes,readable_metrics
0,s3://warehouse/bootcamp/match_details_bucketed/data/match_id_bucket=6/00000-9799-c6e5223a-0d53-4b9e-aa65-ddc616062327-0-00004.parquet,PARQUET,0,Row(match_id_bucket=6),9377,121677,"{1: 29615, 2: 77986, 3: 6550, 4: 6118}","{1: 9377, 2: 9377, 3: 9377, 4: 9377}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: bytearray(b'004acec5-dced-4d'), 2: bytearray(b'A 2 tailed fox'), 3: bytearray(b'\x00\x00\x00\x00'), 4: bytearray(b'\x00\x00\x00\x00')}","{1: bytearray(b'ff28b78b-4073-4:'), 2: bytearray(b'zzzKusohakokids'), 3: bytearray(b'=\x00\x00\x00'), 4: bytearray(b';\x00\x00\x00')}",None,[4],None,0,None,None,None,"Row(match_id=Row(column_size=29615, value_count=9377, null_value_count=0, nan_value_count=None, lower_bound='004acec5-dced-4d', upper_bound='ff28b78b-4073-4:'), player_gamertag=Row(column_size=77986, value_count=9377, null_value_count=0, nan_value_count=None, lower_bound='A 2 tailed fox', upper_bound='zzzKusohakokids'), player_total_deaths=Row(column_size=6118, value_count=9377, null_value_count=0, nan_value_count=None, lower_bound=0, upper_bound=59), player_total_kills=Row(column_size=6550, value_count=9377, null_value_count=0, nan_value_count=None, lower_bound=0, upper_bound=61))"
0,s3://warehouse/bootcamp/match_details_bucketed/data/match_id_bucket=12/00000-9799-c6e5223a-0d53-4b9e-aa65-ddc616062327-0-00001.parquet,PARQUET,0,Row(match_id_bucket=12),9679,125048,"{1: 30496, 2: 80204, 3: 6577, 4: 6367}","{1: 9679, 2: 9679, 3: 9679, 4: 9679}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: bytearray(b'00592b3d-ae26-45'), 2: bytearray(b'A American Monk'), 3: bytearray(b'\x00\x00\x00\x00'), 4: bytearray(b'\x00\x00\x00\x00')}","{1: bytearray(b'fff9b2dc-1391-4e'), 2: bytearray(b'zxXDEATH3Xxz'), 3: bytearray(b':\x00\x00\x00'), 4: bytearray(b'/\x00\x00\x00')}",None,[4],None,0,None,None,None,"Row(match_id=Row(column_size=30496, value_count=9679, null_value_count=0, nan_value_count=None, lower_bound='00592b3d-ae26-45', upper_bound='fff9b2dc-1391-4e'), player_gamertag=Row(column_size=80204, value_count=9679, null_value_count=0, nan_value_count=None, lower_bound='A American Monk', upper_bound='zxXDEATH3Xxz'), player_total_deaths=Row(column_size=6367, value_count=9679, null_value_count=0, nan_value_count=None, lower_bound=0, upper_bound=47), player_total_kills=Row(column_size=6577, value_count=9679, null_value_count=0, nan_value_count=None, lower_bound=0, upper_bound=58))"
0,s3://warehouse/bootcamp/match_details_bucketed/data/match_id_bucket=13/00000-9799-c6e5223a-0d53-4b9e-aa65-ddc616062327-0-00003.parquet,PARQUET,0,Row(match_id_bucket=13),9785,126191,"{1: 30863, 2: 80768, 3: 6699, 4: 6473}","{1: 9785, 2: 9785, 3: 9785, 4: 9785}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: bytearray(b'0019da4a-125b-4d'), 2: bytearray(b'A BALLS86'), 3: bytearray(b'\x00\x00\x00\x00'), 4: bytearray(b'\x00\x00\x00\x00')}","{1: bytearray(b'ffddb9c6-8c61-48'), 2: bytearray(b'zzRICKEYzz'), 3: bytearray(b'G\x00\x00\x00'), 4: bytearray(b'@\x00\x00\x00')}",None,[4],None,0,None,None,None,"Row(match_id=Row(column_size=30863, value_count=9785, null_value_count=0, nan_value_count=None, lower_bound='0019da4a-125b-4d', upper_bound='ffddb9c6-8c61-48'), player_gamertag=Row(column_size=80768, value_count=9785, null_value_count=0, nan_value_count=None, lower_bound='A BALLS86', upper_bound='zzRICKEYzz'), player_total_deaths=Row(column_size=6473, value_count=9785, null_value_count=0, nan_value_count=None, lower_bound=0, upper_bound=64), player_total_kills=Row(column_size=6699, value_count=9785, null_value_count=0, nan_value_count=None, lower_bound=0, upper_bound=71))"
0,s3://warehouse/bootcamp/match_details_bucketed/data/match_id_bucket=1/00000-9799-c6e5223a-0d53-4b9e-aa65-ddc616062327-0-00002.parquet,PARQUET,0,Row(matc